# Bag of Words

## Libraries

In [8]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tiffanyjaya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/tiffanyjaya/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# our libraries 
import tweets_processor

# general libraries 
import json
import nltk
import numpy as np
import re

# feature selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# model selection 
from sklearn.model_selection import train_test_split

# model definition 
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier

# model metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from collections import defaultdict

## Models

In [2]:
def bag_of_words_model(features, target):  
  """A bag-of-words model. Note it disregards the word order in the text."""  
  target = tf.one_hot(target, 15, 1, 0)  
  features = tf.contrib.layers.bow_encoder(      
      features, vocab_size=n_words, embed_dim=EMBEDDING_SIZE)  
  logits = tf.contrib.layers.fully_connected(features, 15,
      activation_fn=None)  
  loss = tf.contrib.losses.softmax_cross_entropy(logits, target)
  train_op = tf.contrib.layers.optimize_loss(
      loss, tf.contrib.framework.get_global_step(),      
      optimizer='Adam', learning_rate=0.01)  
  return (      
      {'class': tf.argmax(logits, 1), 
       'prob': tf.nn.softmax(logits)},      
      loss, train_op)

In [ ]:
classifier = learn.Estimator(model_fn=bag_of_words_model) 
# Train and predict 
classifier.fit(x_train, y_train, steps=10000) 
y_predicted = [ p[‘class’] for p in 
  classifier.predict(x_test, as_iterable=True)] 
score = metrics.accuracy_score(y_test, y_predicted) 
print(‘Accuracy: {0:f}’.format(score))

In [3]:
tweets_text, tweets_place = tweets_processor.get_tweets_from_csv()

In [1]:
import emoji
s = 'test the 🤔 🙈 emojis, in 😌 here 💕👭'
s = ''.join(c for c in s if c not in emoji.UNICODE_EMOJI)
print(s)

ModuleNotFoundError: No module named 'emoji'

In [9]:
#vectorizer = TfidfVectorizer(preprocessor = preprocessor, tokenizer=tokenizer)
# using bigrams
vectorizer = TfidfVectorizer(preprocessor = tweets_processor.preprocessor, tokenizer=tweets_processor.tokenizer,ngram_range=(2,2))
tweets_vectorized = vectorizer.fit_transform(tweets_text)

# length of total vocabulary
#print(len(vectorizer.vocabulary_))
print(tweets_vectorized.shape)


# count vectorizer
#count_vectorizer = CountVectorizer(preprocessor = preprocessor, tokenizer=tokenizer)
#using bigrams
count_vectorizer = CountVectorizer(preprocessor = tweets_processor.preprocessor, tokenizer=tweets_processor.tokenizer,ngram_range=(2,2))
tweets_count_vectorized = count_vectorizer.fit_transform(tweets_text)

# length of total vocabulary
#print(count_vectorizer.vocabulary_)
#print(count_vectorizer.get_feature_names())
#print(tweets_vectorized.shape)

(40001, 233794)


In [12]:
# splitting the data into train, dev, test
#train_data, train_labels = tweets_text[:700], tweets_place[:700] # ????see if we automatically split it proportinately irrespective of size 
#dev_data, dev_labels = tweets_text[700:901], tweets_place[700:901]
#test_data, test_labels = tweets_text[901:], tweets_place[901:]
print(tweets_text[1], tweets_vectorized[1], vectorizer.get_feature_names()[31054], vectorizer.get_feature_names()[1452], vectorizer.get_feature_names()[2172], tweets_place[1])

# using train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(tweets_vectorized, tweets_place, test_size=0.33, random_state=0)

#using count vectorized tweets
#train_data, test_data, train_labels, test_labels = train_test_split(tweets_count_vectorized, tweets_place, test_size=0.33, random_state=0)

does anyone live in the Charleston area and want a job ($10/hr) for Nov 14-21? hmu   (0, 8844)	0.36171867892689485
  (0, 116112)	0.36171867892689485
  (0, 32675)	0.36171867892689485
  (0, 10298)	0.3482676037537245
  (0, 220606)	0.338723914037543
  (0, 102877)	0.3313212531843311
  (0, 95934)	0.36171867892689485
  (0, 140311)	0.36171867892689485 cause much actually radical adventure alaska 3


In [13]:
train_data[0]

<1x233794 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [14]:
# training the model using logisticRegression with multinomial
####???? if there is a better library to use and a better classifier to use, should i svm, ridge classifier?

# Logistic Regression
lg = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
lg.fit(train_data, train_labels)

print("Accuracy on test set: {:.02%}".format(lg.score(test_data, test_labels)))
# print(lg.score(test_data, test_labels))

#Ridge Classifier
rc = RidgeClassifier()
rc.fit(train_data, train_labels)

pred = rc.predict(test_data)

print("Accuracy on test set: {:.02%}".format(rc.score(test_data, test_labels)))
#print(rc.score(test_data, test_labels))
#print(classification_report(test_labels, pred))


/home/tiffanyjaya/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Accuracy on test set: 26.95%
Accuracy on test set: 27.26%


In [15]:
# using naive bayes
nb = MultinomialNB()
nb.fit(train_data, train_labels)
y_pred = nb.predict(test_data)
acc = accuracy_score(test_labels, y_pred)
print("Accuracy on test set: {:.02%}".format(acc))

Accuracy on test set: 26.83%


In [ ]:
# using MLP classifier
mlp = MLPClassifier(hidden_layer_sizes=(100, ),)
mlp.fit(train_data, train_labels)
y_pred = mlp.predict(test_data)
acc = accuracy_score(test_labels, y_pred)
print("Accuracy on test set: {:.02%}".format(acc))

<h4>Logic for the baseline model to get words for each region

In [ ]:
print(tweets_text[0], tweets_vectorized[1], vectorizer.get_feature_names()[1485], vectorizer.get_feature_names()[1452], vectorizer.get_feature_names()[2172], tweets_place[0])
vocab = vectorizer.get_feature_names()
place_word_score = defaultdict(lambda: defaultdict(lambda: 0.0))
word_place_score = defaultdict(lambda: defaultdict(lambda: 0.0))
word_score = dict()
for index, tweet in enumerate(tweets_vectorized):
    place = tweets_place[index]
    for word_index, score in zip(tweet.indices, tweet.data):
        word = vocab[word_index]
        existing_place = place_word_score.get(place, None)
        if existing_place is not None and existing_place.get(word, None) is not None:
            score = existing_place.get(word) + score
        # stores for each place what are all the words
        place_word_score[place][word] = score
        word_score_in_dict = word_score.get(word, None)
        # update the dict with the place and word only when there is no word in dict(word_score) with a score or if the score is greater for this place than the previous place
        # also update the word_score with new score
        if word_score_in_dict is None or word_score_in_dict < score:
            word_place_score[word] = {place:score}
            word_score[word] = score
print(word_place_score)